# HELIPADCAT: CATEGORISED HELIPAD IMAGE DATASET AND DETECTION METHOD

# 1. Database Creation


Based on the FAA’s database of US airports, we create the first dataset of helipads, including 
a classification by visual helipad shape and features, which we make available to the research community. 
The dataset includes nearly 6,000 images with 12 different categories.

In the `data` folder is a csv file containing the dataset. The object `src.databse_management.csv_to_meta`
creates the annotated dataset and downloads the images from Google Maps.

In [ ]:
from src.database_management.csv_to_meta import CsvToMeta

input_csv_file_path = "data//Helipad_DataBase_annotated.csv"
output_folder = "data"
download_images = False
google_api_key_filepath = None

csv_to_meta = CsvToMeta(input_csv_file_path=input_csv_file_path,
                       output_folder=output_folder,
                       download_images=download_images,
                       google_api_key_filepath=google_api_key_filepath)

csv_to_meta.run()

Once the dataset created, we can start the next step.

# 2. Build Groundtruth
Annotate each image by drawing a bounding boxes around the helipads in the images.
This step has already been done.

In [ ]:
from src.database_management.build_groundtruth import GroundTruth

database_folder = "C:\\Users\\jonas\\Desktop\\Detection\\Detection_Dataset"
meta_folder = "C:\\Users\\jonas\\Desktop\\Detection\\Detection_Dataset_meta"

ground_truth = GroundTruth(database_folder,
                           meta_folder,
                           review=False,
                           augment_only=False,
                           redo_false=False,
                           redo_true=True,
                           tms_dataset=True)

ground_truth.run()

# 3. Assign Categories

Assign a pattern category to each helipad. This step has already been done.

In [ ]:
from src.database_management.database_categories_v2 import DatabaseCategoriesv2

image_folder = os.path.join('C:\\', 'Users', 'jonas', 'Desktop', 'Helipad', 'Helipad_DataBase', 'Helipad_DataBase_original')
meta_folder = os.path.join('C:\\', 'Users', 'jonas', 'Desktop', 'Helipad', 'Helipad_DataBase_meta', 'Helipad_DataBase_meta_original')

database_categories = DatabaseCategoriesv2(image_folder, meta_folder, nb_categories=12)

database_categories.build_categories()

# 4. DataBase Augmentation

Apply Image augmentation on the images using Google's policy (`src.database_management.Database_augmentation`) or custom made policy with ImgAug (`src.database_management.database_augmentation_v2`)

### 4.1 Using Google's Policy

In [ ]:
from src.database_management.Database_augmentation import DatabaseAugmentation

input_folder = "data//Helipad_DataBase//Helipad_DataBase_original"
meta_folder = "data//Helipad_DataBase_meta//Helipad_DataBase_meta_original"
root_folder = "data//Helipad_DataBase"
root_folder_meta = "data//Helipad_DataBase_meta"

version_number = 12

balance_dataset = True

repartition = [3, 1, 1, 3, 0, 3, 2, 0, 1, 1, 0, 0]

database_augmentation = DatabaseAugmentation(input_folder,
                                             meta_folder,
                                             root_folder,
                                             root_folder_meta,
                                             balance_dataset=balance_dataset,
                                             repartition=repartition,
                                             version_number=version_number,
                                             display=False)

database_augmentation.run()

### 4.2 Using Custom Policy

In [ ]:
from imgaug import augmenters as iaa
from src.database_management.database_augmentation_v2 import DatabaseAugmentationV2

input_folder = "data//Helipad_DataBase//Helipad_DataBase_original"
meta_folder = "data//Helipad_DataBase_meta//Helipad_DataBase_meta_original"
root_folder = "data//Helipad_DataBase"
root_folder_meta = "data//Helipad_DataBase_meta"

augmentation_strategy = iaa.Sequential([
                            iaa.Sometimes(0.2, iaa.Fliplr(1)),
                            iaa.Sometimes(0.2, iaa.Flipud(1)),
                            iaa.Sometimes(0.2, iaa.Rotate((-90, 90))),
                            iaa.Sometimes(0.2, iaa.Affine(scale=(0.5, 1.5))),
                            iaa.Sometimes(0.2, iaa.GaussianBlur(sigma=(0.0, 3.0))),
                            iaa.Sometimes(0.15, iaa.AllChannelsHistogramEqualization()),
                            iaa.Sometimes(0.2, iaa.ShearX((-20, 20))),
                            iaa.Sometimes(0.2, iaa.ShearY((-20, 20))),
                            iaa.Sometimes(0.15, iaa.pillike.EnhanceSharpness()),
                            iaa.Sometimes(0.15, iaa.pillike.EnhanceBrightness())
                            ])

version_number = 11

balance_dataset = True

repartition = [3, 1, 1, 3, 0, 3, 2, 0, 1, 1, 0, 0]

database_augmentation = DatabaseAugmentationV2(input_folder,
                                             meta_folder,
                                             root_folder,
                                             root_folder_meta,
                                             augmentation_strategy=augmentation_strategy,
                                             version_number=version_number,
                                             balance_dataset=balance_dataset,
                                             repartition=repartition)

database_augmentation.run()


# 5. Run Training

Run a training with `src.training.run_training` by specifying the dataset root folder, the model folder, tha augmented version, the train and test categories and the starting model weights.

The training framework MRCNN has to be installed. See : <https://github.com/matterport/Mask_RCNN>

Once the training has started, it is possible to keep track of the evolution of the metrics with Tensorboard. Simply launch Tensorboard with the command : `tensorboard --logdir model_dir_path --port 8890`.

In [ ]:
from src.training.run_training import RunTraining

root_folder = "data//Helipad_DataBase"
root_meta_folder = "data//Helipad_DataBase_meta"
model_folder = "model"
include_augmented = True
augmented_version = [10]

train_categories = ["1", "2", "3", "5", "6", "8", "9"]
test_categories = ["4", "7", "d", "u"]

weights_filename = 'helipad_cfg_10_no47du_all20200420T0127/mask_rcnn_helipad_cfg_10_no47du_all_0538.h5'
base_weights = 'mask_rcnn_coco.h5'

predict_weights_filepath = 'helipad_cfg_7_aug123_all20200106T2012/mask_rcnn_helipad_cfg_7_aug123_all_0472.h5'

run_training = RunTraining(root_folder,
                           root_meta_folder,
                           model_folder,
                           weights_filename,
                           include_augmented=include_augmented,
                           augmented_version=augmented_version,
                           predict_weights_filepath=None,
                           train_categories=train_categories,
                           test_categories=test_categories)

print('Starting Training')
run_training.run()
print('Training Over')

# 6. Evaluate mAP on original dataset

This step compute the mAP on the original dataset to get a first metric on the performances of the newly trained model. The object used is the same as in the previous step. The method to execute is : `RunTraining.run_predict()`. 

In [ ]:
from src.training.run_training import RunTraining

root_folder = "data//Helipad_DataBase"
root_meta_folder = "data//Helipad_DataBase_meta"
model_folder = "model"
include_augmented = False
augmented_version = []

train_categories = ["1", "2", "3", "5", "6", "8", "9"]
test_categories = ["1", "2", "3", "5", "6", "8", "9"]

weights_filename = 'helipad_cfg_10_no47du_all20200420T0127/mask_rcnn_helipad_cfg_10_no47du_all_0851.h5'
base_weights = 'mask_rcnn_coco.h5'

predict_weights_filepath = 'helipad_cfg_12_no47du_all20200513T1024/mask_rcnn_helipad_cfg_12_no47du_all_0260.h5'

run_training = RunTraining(root_folder,
                           root_meta_folder,
                           model_folder,
                           weights_filename,
                           include_augmented=include_augmented,
                           augmented_version=augmented_version,
                           predict_weights_filepath=predict_weights_filepath,
                           train_categories=train_categories,
                           test_categories=test_categories)

print('Evaluating Last Epoch')
run_training.run_predict()
print('Evaluation Done')

# 7. Run Detection on Original Dataset to save bounding boxes

Here, with `src.detection.run_detection`, the images are feeded into the network and the bounding boxes 
are saved inside the meta files.

In [ ]:
from src.detection.run_detection import RunDetection

image_folder = "data//Helipad_DataBase//Helipad_DataBase_original"
meta_folder = "data//Helipad_DataBase_meta//Helipad_DataBase_meta_original"
model_root_folder = "model"
model_folder = "helipad_cfg_12_no47du_all20200513T1024"
model_number = 12
weight_filename = "mask_rcnn_helipad_cfg_12_no47du_all_0260.h5"

test_only = False
activate_filter = False

run_detection = RunDetection(image_folder,
                          meta_folder,
                          os.path.join(model_root_folder, model_folder),
                          weight_filename,
                          model_number=model_number,
                          activate_filter=activate_filter,
                          test_only=test_only)

run_detection.run()

# 8. Run Benchmark on a model

Once the bounding boxes have been saved, the accuracy, error, precision and recall are computed with `src.benchmark.run_benchmark`. A csv file is created inside the folder `src/benchmark/benchmark_results/` containing the results for a wide range of score threshold.

### 8.1 Benchmark on Train Set

In [ ]:
from src.benchmark.run_benchmark import RunBenchmark

image_folder = "data//Helipad_DataBase//Helipad_DataBase_original"
meta_folder = "data//Helipad_DataBase_meta//Helipad_DataBase_meta_original"
model_numbers = [12]
test_only = False
tms_dataset = False
zoom_level = None
city_lat = None
include_negative = False
train_only = True
train_categories = ["1", "2", "3", "5", "6", "8", "9"]

run_benchmark = RunBenchmark(image_folder,
                             meta_folder,
                             model_numbers,
                             test_only=test_only,
                             tms_dataset=tms_dataset,
                             include_category=train_categories,
                             zoom_level=zoom_level,
                             city_lat=city_lat,
                             train_only=train_only)

threshold_validation = None

run_benchmark.run(threshold_validation=threshold_validation)

### 8.2 Benchmark on Test Set

In [ ]:
from src.benchmark.run_benchmark import RunBenchmark

image_folder = "data//Helipad_DataBase//Helipad_DataBase_original"
meta_folder = "data//Helipad_DataBase_meta//Helipad_DataBase_meta_original"
model_numbers = [12]
test_only = True
tms_dataset = False
zoom_level = None
city_lat = None
test_categories = ["1", "2", "3", "5", "6", "8", "9"]
include_negative = True
train_only = False


run_benchmark = RunBenchmark(image_folder,
                             meta_folder,
                             model_numbers,
                             test_only=test_only,
                             tms_dataset=tms_dataset,
                             include_category=test_categories,
                             include_negative=include_negative,
                             zoom_level=zoom_level,
                             city_lat=city_lat,
                             train_only=train_only)

threshold_validation = None

run_benchmark.run(threshold_validation=threshold_validation)

# 9. Run Prediction on Satellite Images

In this step, with `src.detection.run_prediction_satellite`, additional unseen data are feeded into the network to detect helipads in any area of the world. 
First, the additional images have to be downloaded with SAS Planet software and store into a cache folder following the TMS file structure, preferably with a zoom of 19. 
The bounding boxes are saved into meta files. The predictions will be vizualised on the map after our custom filtering. 

In [ ]:
from src.detection.run_prediction_satellite import RunPredictionSatellite

cache_tms_sat_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS\\sat"
output_meta_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS_meta\\sat"
model_folder = "model"

weights_filename = "helipad_cfg_12_no47du_all20200513T1024/mask_rcnn_helipad_cfg_12_no47du_all_0260.h5"
model_number = 12

zoom_level = 19
activate_filters = False

run_prediction_satellite = RunPredictionSatellite(cache_tms_sat_folder=cache_tms_sat_folder,
                                                  output_meta_folder=output_meta_folder,
                                                  zoom_level=zoom_level,
                                                  model_folder=model_folder,
                                                  weights_filename=weights_filename,
                                                  model_number=model_number,
                                                  activate_filters=activate_filters)

run_prediction_satellite.run()

# 10. Build index path by score on Satellite Images

When the area to scan is big, the number of images to feed into the network is high. After the detection, since the number of helipads is very low compared to the number of images, a text file is created, serving as an index file, having in each line the location of the meta file with a bounding box inside. The object `src.database_management.index_path_by_score` allows such indexation.

In [ ]:
from src.database_management.index_path_by_score import IndexPathScore

meta_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS_meta\\sat\\19\\"
model_number = 7
score_threshold = 0

index_path_score = IndexPathScore(meta_folder, model_number, score_threshold)

index_path_score.run()

index_path_score.write_output()

# 11. Optional Step : Train and Run Validation on detected bounding boxes using KNN

The idea behind this step is to have a KNN who validates the detected bounding boxes as true or false. This approach did not give good performance.

### 11.1 Build the dataset of bounding boxes

In [ ]:
from src.knn.knn_build_database import KNNBuildDatabase

image_folder = "data//Helipad_DataBase//Helipad_DataBase_original"
meta_folder = "data//Helipad_DataBase_meta//Helipad_DataBase_meta_original"
model_number = 7

knn_build_database = KNNBuildDatabase(image_folder, meta_folder, model_number)

knn_build_database.run()

### 11.2 Train a KNN on this dataset

In [ ]:
from src.knn.knn_training import KNNTraining

knn_training = KNNTraining(nb_neighbors=2, nb_jobs=-1, test_size=0.25)
knn_training.fit(knn_build_database.X, knn_build_database.y, mode="histogram", binary=True)
knn_training.score()
knn_training.save(model_number=model_number, dataset='original')

### 11.3 Predict on new images

In [ ]:
from knn.knn_predict import KNNPredict

image_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS\\sat"
meta_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS_meta\\sat"
index_path_score_filename = "C:\\Users\\AISG\\Documents\\Jonas\\helipad_detection\\src\\database_management\\helipad_path_over_0.999.txt"
model_number = 7
knn_weights_filename = "knn_histogram_2.pkl"
random_forest_weights_filename = "random_forest_e100.pkl"

model = "knn"

knn_predict = KNNPredict(image_folder,
                         meta_folder,
                         index_path_score_filename,
                         model_number,
                         knn_weights_filename,
                         model)

knn_predict.predict()

knn_predict.write_prediction_to_meta()

# 12. Optional Step : Run Validation on detected bounding boxes using CNN

The idea behind this step is to have another network who validates the detected bounding boxes as true or false. This approach did not give good performance.

### 12.1 Build the dataset of bounding boxes

In [ ]:
from src.bb_analysis.bb_build_dataset import BBBuildDataset

image_folder = "data//Helipad_DataBase//Helipad_DataBase_original"
meta_folder = "data//Helipad_DataBase_meta//Helipad_DataBase_meta_original"
model_number = 10
score_threshold = 0.0
iou_threshold = 0.5
output_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Helipad\\Detected_Boxes\\"
tms = False
# ground_truth_bb indicates wheter the dataset keeps the groundtruth
# as the true positive (True) or the true positive detected by the model (False)
# Sometime, the detection by the model is not centered in the helipad
# hence it can add noise to the model
groundtruth_bb = True
# the bounding boxes of those categories will be considered as false positive
filter_categories = ["4", "7", "d", "u"]
balance_categories = True

bb_build_dataset = BBBuildDataset(image_folder=image_folder,
                                  meta_folder=meta_folder,
                                  model_number=model_number,
                                  score_threshold=score_threshold,
                                  iou_threshold=iou_threshold,
                                  output_folder=output_folder,
                                  tms=tms,
                                  groundtruth_bb=groundtruth_bb,
                                  filter_categories=filter_categories,
                                  balance_categories=balance_categories)

bb_build_dataset.run()

### 12.2 Build Bounding Box Dataset on Real World images

In [ ]:
from src.bb_analysis.bb_build_dataset import BBBuildDataset

image_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS\\sat\\19"
meta_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS_meta\\sat\\19"
model_number = 10
score_threshold = 0.0
iou_threshold = 0.5
output_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Helipad\\Real_World_Detected_Boxes\\"
tms = True
# ground_truth_bb indicates wheter the dataset keeps the groundtruth
# as the true positive (True) or the true positive detected by the model (False)
# Sometime, the detection by the model is not centered in the helipad
# hence it can add noise to the model
groundtruth_bb = False
# the bounding boxes of those categories will be considered as false positive
filter_categories = None
balance_categories = False
zoom_out = 5
index_path = "helipad_detection\\src\\helipad_path_over_0_m10.txt"

bb_build_dataset = BBBuildDataset(image_folder=image_folder,
                                  meta_folder=meta_folder,
                                  model_number=model_number,
                                  score_threshold=score_threshold,
                                  iou_threshold=iou_threshold,
                                  output_folder=output_folder,
                                  tms=tms,
                                  groundtruth_bb=groundtruth_bb,
                                  filter_categories=filter_categories,
                                  balance_categories=balance_categories,
                                  zoom_out=zoom_out,
                                  index_path=index_path)

bb_build_dataset.run()

### 12.3 Train a CNN Model

In [ ]:
from bb_analysis.bb_training_manager import BBTrainingManager

image_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Helipad\\Detected_Boxes\\model_10_0.0"
model_filename = "bb_model_adam_64.h5"

bbtraining_manager = BBTrainingManager(image_folder)

bbtraining_manager.run()

bbtraining_manager.evaluate()

bbtraining_manager.save(filename=model_filename)

bbtraining_manager.plot()

### 12.4 Predict on the original dataset 

In [ ]:
from bb_analysis.bb_predict import BBPredict

image_folder = "data//Helipad_DataBase//Helipad_DataBase_original"
meta_folder = "data//Helipad_DataBase_meta//Helipad_DataBase_meta_original"
model_number = 7
model_path = "final_model.h5"
tms = False
extract_bounding_boxes = True

bbpredict = BBPredict(image_folder=image_folder,
                      meta_folder=meta_folder,
                      model_number=model_number,
                      model_path=model_path,
                      tms=tms,
                      extract_bounding_boxes=extract_bounding_boxes)

bbpredict.run()

### 12.5 Predict on Satellite Images using index path

In [ ]:
import sys
sys.path.append('bb_analysis')
from bb_predict import BBPredict

image_folder = "C:\\Users\\AISG\\Documents\\Jonas\\City\\cebu\\sat"
meta_folder = "C:\\Users\\AISG\\Documents\\Jonas\\City\\cebu_meta\\sat"
model_number = 10
model_path = "bb_analysis/checkpoint_manilla/weights.81-0.01-1.00-0.32-0.94.h5"
tms = True
extract_bounding_boxes = True

index_path = "helipad_path_over_0_m10.txt"
index_path = None

bbpredict = BBPredict(image_folder=image_folder,
                      meta_folder=meta_folder,
                      model_number=model_number,
                      model_path=model_path,
                      tms=tms,
                      extract_bounding_boxes=extract_bounding_boxes,
                      index_path=index_path)

bbpredict.run()

## 13. Compute the area of the bounding boxes on Satellite Images

Our first custom filter computes the ground area of the bounding boxes in meter squared. The areas are saved into the meta files. The object used is `src.bb_analysis.bb_compute_area_tms`.

In [ ]:
from src.bb_analysis.bb_compute_area_tms import BBComputeAreaTMS

image_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS\\sat\\"
meta_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS_meta\\sat\\"
model_number = 10
index_path = "helipad_path_over_0_m10.txt"

bb_compute_area_tms = BBComputeAreaTMS(image_folder=image_folder,
                                       meta_folder=meta_folder,
                                       model_number=model_number,
                                       index_path=index_path)

bb_compute_area_tms.run()

## 14. Apply Shadow Detection on detected bounding boxes

Our second custom filter looks for the presence of shadow inside the bounding boxes. The object used is `src.bb_analysis.run_shadow_detection`. The results are saved into the meta files.

In [ ]:
from src.bb_analysis.run_shadow_detection import RunShadowDetection

image_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS\\sat\\"
meta_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS_meta\\sat\\"
model_number = 10
groundtruth_only = False
tms = True
zoom_out = 5
index_path = "C:\\Users\\AISG\\Documents\\Jonas\\helipad_detection\\src\\helipad_path_over_0_m10.txt"
minimum_size_window = 3
threshold_v = 0.35
threshold_s = 0.02
ratio = 1
d_0 = 3

run_shadow_detection = RunShadowDetection(image_folder=image_folder, 
                                          meta_folder=meta_folder,
                                          model_number=model_number, 
                                          groundtruth_only=groundtruth_only, 
                                          tms=tms, 
                                          zoom_out=zoom_out, 
                                          index_path=index_path,
                                          minimum_size_window=minimum_size_window, 
                                          threshold_v=threshold_v, 
                                          threshold_s=threshold_s, 
                                          ratio=ratio, 
                                          d_0=d_0)

run_shadow_detection.run()

## 15. Optional Step : Build groundtruth on Satellite images

This step allows the user to manually annotated the groundtruth of the detected bounding boxes on additional data in order to compute the performance of the model. The object is `src.database_management.build_groundtruth_tms`. With the index file created in step 10, only the images with a bounding boxes are to be validated manually as true or false. 

In [ ]:
from src.database_management.build_groundtruth_tms import BuildGroundtruthTMS

image_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS\\sat\\"
meta_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS_meta\\sat\\"
model_number = 10
index_path = "C:\\Users\\AISG\\Documents\\Jonas\\helipad_detection\\src\\helipad_path_over_0_m10.txt"
source_from = "C:\\Users\\AISG\\Documents\\Jonas\\Helipad\\Real_World_Detected_Boxes\\model_10_0.0_groundtruth\\"
start_index = 0

build_groundtruth_tms = BuildGroundtruthTMS(image_folder=image_folder,
                                            meta_folder=meta_folder,
                                            model_number=model_number,
                                            index_path=index_path,
                                            source_from=source_from,
                                            start_index=start_index)

build_groundtruth_tms.run()



# 16. Benchmark on Satellite images annotated with groundtruth

After step 15, it is then possible to compute the benchmark on additional data with the object `src.benchmark.benchmark_manager_TMS`. The user can configure the parameters of our three custom filters (shadow, area and score). 

In [ ]:
import json
from src.benchmark.benchmark_manager_TMS import BenchmarkManagerTMS

image_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS\\sat\\"
meta_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS_meta\\sat\\"
model_number = 10
index_path = "C:\\Users\\AISG\\Documents\\Jonas\\helipad_detection\\src\\helipad_path_over_0_m10.txt"

filters_config = { 
              'shadow': {
                  'activate': True,
                  'zoom_out': 5
                },
               'area' : {
                   'activate': True,
                   'lower': 164,
                   'higher': 547,
               },
                'score': {
                    'activate': True,
                    'threshold': 0.99
                },
                'cnn_validation': {
                    'activate': False,
                    'threshold': 0
                }
            }

benchmark_manager_tms = BenchmarkManagerTMS(image_folder=image_folder,
                                            meta_folder=meta_folder,
                                            model_number=model_number,
                                            index_path=index_path)

results = benchmark_manager_tms.run(filters_config)

print(results)

# 17. Build Placemarks

This final step of our method creates a placemarks file in order to visualize the detection on the map. Each center of the bounding boxes are pin points on the map of the world, as shown below. The object used is `src.detection.build_placemarks`. 

In [9]:
from src.detection.build_placemarks import BuildPlacemarks

meta_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS_meta\\sat\\"
model_number = 7
index_path = "C:\\Users\\AISG\\Documents\\Jonas\\helipad_detection\\src\\helipad_path_over_0_m7.txt"

filters_config = { 
              'shadow': {
                  'activate': True,
                  'zoom_out': 5
                },
               'area' : {
                   'activate': True,
                   'lower': 164,
                   'higher': 547,
               },
                'score': {
                    'activate': True,
                    'threshold': 0.99
                },
                'cnn_validation': {
                    'activate': False,
                    'threshold': 0
                }
            }

prefix = "Manilla_"

build_placemarks = BuildPlacemarks(meta_folder=meta_folder,
                                   model_number=model_number,
                                   index_path=index_path,
                                   filters_config=filters_config,
                                   prefix=prefix)

build_placemarks.run()

100%|████████████████████████████████████████████████████████████████████████████| 1863/1863 [00:00<00:00, 9133.46it/s]


### 17.2 Transfer the placemarks to my local machine for visualization

# Save the Original Dataset with groundtruth from Meta to CSV

In [ ]:
import os 
os.chdir('../../')

from helipad_detection.src.database_management.meta_to_csv import MetaToCsv


meta_folder_original = "C:\\Users\\AISG\\Documents\\Jonas\\Helipad\\Helipad_DataBase_meta\\Helipad_DataBase_meta_original"
output_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Helipad\\Helipad_DataBase_meta"
output_filename = "Helipad_DataBase_annotated.csv"

meta_to_csv = MetaToCsv(meta_folder=meta_folder_original,
                        output_folder=output_folder,
                        output_filename=output_filename)

meta_to_csv.run()